# Conexión a la base de datos

In [1]:
import sqlite3

conexion = sqlite3.connect('base.db')
cursor = conexion.cursor()

# Activación de claves foráneas para Sqlite
cursor.execute("PRAGMA foreign_keys = ON")


# Creación de tablas

### Pais

In [2]:
cursor.execute("""
    CREATE TABLE Pais (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL UNIQUE
    )
""")

print("Tabla creada")

Tabla creada


### Producto

In [3]:
cursor.execute("""
    CREATE TABLE Producto (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL UNIQUE
    )
""")

print("Tabla creada")

Tabla creada


### Cliente

In [7]:
cursor.execute("""
    CREATE TABLE Cliente (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL,
        apellido TEXT NOT NULL,
        nacimiento TEXT,
        pais_origen_id INTEGER REFERENCES Pais(id) ON DELETE SET NULL
        )
""")

print("Tabla creada")

Tabla creada


### Cliente_Producto

In [4]:
cursor.execute("""
    CREATE TABLE Cliente_Producto (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        cliente_id INTEGER REFERENCES Cliente(id) ON DELETE CASCADE,
        producto_id INTEGER REFERENCES Producto(id) ON DELETE CASCADE
        )
""")

print("Tabla creada")

Tabla creada


# Creación de registros

### Paises

In [5]:
sql = "INSERT INTO Pais (nombre) VALUES (?)"
cursor.execute(sql, ("Argentina",))
conexion.commit()
print("Paises creados")


Paises creados


In [6]:
cursor.execute("SELECT * FROM Pais")
print(cursor.fetchall())

[(1, 'Argentina')]


In [11]:
paises = ["Brasil", "Chile", "México", "Ecuador", "Colombia", "Uruguay"]
for pais in paises:
    cursor.execute(sql, (pais,))
conexion.commit()
print("Paises creados")

Paises creados


In [12]:
cursor.execute("SELECT * FROM Pais")
print(cursor.fetchone())

(1, 'Argentina')


In [11]:
cursor.execute("SELECT * FROM Pais")
print(cursor.fetchmany(3))

[(1, 'Argentina'), (2, 'Brasil'), (3, 'Chile')]


### Productos

In [12]:
productos = [("azúcar",), ("leche",), ("queso",)]

sql = "INSERT INTO Producto (nombre) VALUES (?)"
cursor.executemany(sql, productos)
print("Productos creados")

Productos creados


In [13]:
cursor.execute("SELECT * FROM Producto")
items = cursor.fetchall()
for item in items:
    print(item[0], item[1])

1 azúcar
2 leche
3 queso


### Cliente

In [7]:
print("CREANDO CLIENTE")

nombre = input("Nombre: ")
if nombre == "":
    nombre = None
apellido = input("Apellido: ")
if apellido == "":
    apellido = None
nacimiento = input("Fecha de nacimiento (aaaa-mm-dd): ")

# SELECT
print("País de origen:")
cursor.execute("SELECT * FROM Pais")
items = cursor.fetchall()
for item in items:
    print(f"\t{item[0]}: {item[1]}")

while True:
    entrada = int(input("Seleccione opción: "))
    if entrada in [x[0] for x in items]:
        break
    else:
        continue
pais_origen_id = entrada

# INSERT
sql = "INSERT INTO Cliente (nombre, apellido, nacimiento, pais_origen_id) VALUES (?,?,?,?)"
cursor.execute(sql, (nombre, apellido, nacimiento, pais_origen_id))
conexion.commit()
print("Cliente creado")

CREANDO CLIENTE
País de origen:
	1: Argentina


OperationalError: no such table: Cliente

In [15]:
cursor.execute("SELECT * FROM Cliente")
print(cursor.fetchall())

[(1, 'Esteban', 'Acevedo', '1981-1-1', 1)]


In [16]:
cursor.execute("""
            SELECT Cliente.id, Cliente.nombre, apellido, nacimiento, Pais.nombre
            FROM Cliente INNER JOIN Pais ON Cliente.pais_origen_id = Pais.id
        """)
items = cursor.fetchall()

encabezado = f"{'ID':<5} | {'NOMBRE':<10} | {'APELLIDO':<15} | {'NACIMIENTO':<15} | {'PAIS ORIGEN':<15}"
print(encabezado + "\n" + len(encabezado) * "=")
for item in items:
    print(f"{item[0]:<5} | {item[1]:<10} | {item[2]:<15} | {item[3]:<15} | {item[4]:<15}")

ID    | NOMBRE     | APELLIDO        | NACIMIENTO      | PAIS ORIGEN    
1     | Esteban    | Acevedo         | 1981-1-1        | Argentina      


### Cliente_Producto

In [ ]:
print("CLIENTE COMPRA PRODUCTO")

# SELECT
print("Clientes:")
cursor.execute("SELECT * FROM Cliente")
items = cursor.fetchall()
for item in items:
    print(f"\t{item[0]}: {item[1]} {item[2]}")

while True:
    entrada = int(input("Seleccione opción: "))
    if entrada in [x[0] for x in items]:
        break
    else:
        continue
if entrada == "":
    entrada = None
cliente_id = entrada

print("Productos:")
cursor.execute("SELECT * FROM Producto")
items = cursor.fetchall()
for item in items:
    print(f"\t{item[0]}: {item[1]}")

while True:
    entrada = int(input("Seleccione opción: "))
    if entrada in [x[0] for x in items]:
        break
    else:
        continue
if entrada == "":
    entrada = None
producto_id = entrada

# INSERT
sql = "INSERT INTO Cliente_Producto (cliente_id, producto_id) VALUES (?,?)"
cursor.execute(sql, (cliente_id, producto_id))
conexion.commit()
print("Cliente - Producto creado")

In [ ]:
cursor.execute("SELECT * FROM Cliente_Producto")
print(cursor.fetchall())

In [ ]:
cursor.execute("""
            SELECT Cliente_Producto.id, Cliente.nombre, Cliente.apellido, Producto.nombre
            FROM Cliente_Producto
            INNER JOIN Cliente ON Cliente_Producto.cliente_id = Cliente.id
            INNER JOIN Producto ON Cliente_Producto.producto_id = Producto.id
        """)
items = cursor.fetchall()

encabezado = f"{'ID':<5} | {'CLIENTE':<25} | {'PRODUCTO':<15}"
print(encabezado + "\n" + len(encabezado) * "=")
for item in items:
    cliente = item[1] + " " + item[2]
    print(f"{item[0]:<5} | {cliente:<25} | {item[3]:15}")

# Consultas con condiciones

In [ ]:
cursor.execute("""
            SELECT Cliente_Producto.id, Cliente.nombre, Cliente.apellido, Producto.nombre
            FROM Cliente_Producto
            INNER JOIN Cliente ON Cliente_Producto.cliente_id = Cliente.id
            INNER JOIN Producto ON Cliente_Producto.producto_id = Producto.id
            ORDER BY Producto.nombre ASC
        """)
items = cursor.fetchall()
for item in items:
    print(item)

In [ ]:
cursor.execute("""
            SELECT Cliente_Producto.id, Cliente.nombre, Cliente.apellido, Producto.nombre
            FROM Cliente_Producto
            INNER JOIN Cliente ON Cliente_Producto.cliente_id = Cliente.id
            INNER JOIN Producto ON Cliente_Producto.producto_id = Producto.id
            WHERE Producto.nombre = "queso"
        """)
items = cursor.fetchall()
for item in items:
    print(item)

In [ ]:
cursor.execute("""
            SELECT Cliente_Producto.id, Cliente.nombre, Cliente.apellido, Producto.nombre
            FROM Cliente_Producto
            INNER JOIN Cliente ON Cliente_Producto.cliente_id = Cliente.id
            INNER JOIN Producto ON Cliente_Producto.producto_id = Producto.id
            WHERE Cliente.nombre like "%e%"
        """)
items = cursor.fetchall()
for item in items:
    print(item)

# Modificación de registros

In [ ]:
print("Modificando cliente.")
apellido_cambiar = input("Ingrese apellido a cambiar: ")
apellido_nuevo = input("Ingrese apellido nuevo: ")

sql = "UPDATE Cliente SET apellido = ? WHERE apellido = ?"
cursor.execute(sql, (apellido_nuevo, apellido_cambiar))
conexion.commit()
print("Registro modificado")

In [ ]:
cursor.execute("SELECT * FROM Cliente")
items = cursor.fetchall()
for item in items:
    print(item)

# Eliminación de registros

In [ ]:
print("Eliminando producto.")
producto_eliminar = input("Ingrese producto a eliminar: ")

sql = "DELETE FROM Producto WHERE nombre = ?"
cursor.execute(sql, (producto_eliminar,))
conexion.commit()
print("Registro eliminado")

In [ ]:
cursor.execute("SELECT * FROM Producto")
items = cursor.fetchall()
for item in items:
    print(item)

In [ ]:
sql = "DELETE FROM Pais WHERE nombre = ?"
cursor.execute(sql, ("Argentina",))
conexion.commit()
print("Registro eliminado")

In [ ]:
cursor.execute("SELECT * FROM Pais")
items = cursor.fetchall()
for item in items:
    print(item)

In [ ]:
cursor.execute("SELECT * FROM Cliente")
items = cursor.fetchall()
for item in items:
    print(item)

In [ ]:
cursor.execute("SELECT * FROM Cliente_Producto")
items = cursor.fetchall()
for item in items:
    print(item)



In [ ]:
cursor.execute("""
            SELECT Cliente_Producto.id, Cliente.nombre, Cliente.apellido, Producto.nombre
            FROM Cliente_Producto
            INNER JOIN Cliente ON Cliente_Producto.cliente_id = Cliente.id
            INNER JOIN Producto ON Cliente_Producto.producto_id = Producto.id
        """)
items = cursor.fetchall()

encabezado = f"{'ID':<5} | {'CLIENTE':<25} | {'PRODUCTO':<15}"
print(encabezado + "\n" + len(encabezado) * "=")
for item in items:
    cliente = item[1] + " " + item[2]
    print(f"{item[0]:<5} | {cliente:<25} | {item[3]:15}")

Uso de Pandas

In [1]:
import pandas

datos = pandas.read_csv("dataset_turnos_detalle.csv")


In [ ]:
datos

In [ ]:
datos["sede"].value_counts()